# Examples of how to run Barry for DESI KP4
This notebook demonstrates how to use Barry with the KP4 data. 

The data gets pre-processed into a set of 'pickle' files that contain everything Barry needs in the notebook `/barry/data/desi_kp4/pickle_desi_kp4.py`. After running this, you should have the following set of 16 files in `/barry/data/`

* `desi_kp4_abacus_cubicbox_{xi/pk}_lrg.pkl`
* `desi_kp4_ezmock_cubicbox_{xi/pk}_lrg.pkl`
* `desi_kp4_abacus_cutsky_{xi/pk}_lrg_zmin{0.4/0.6/0.8}_zmax{0.6/0.8/1.1}.pkl`
* `desi_kp4_ezmock_cutsky_{xi/pk}_lrg_zmin{0.4/0.6/0.8}_zmax{0.6/0.8/1.1}.pkl`

I anticipate there being more as we introduce more tracers etc., but the code for producing them should be extendable easily enough. This notebook covers the details of how these get read in, how to make some plots with them, and how to run some simple optimisations and sanity checks with different models.

## Reading in the data
I've already created classes for the KP4 data. You can find them in `/barry/datasets/dataset_power_spectrum.py` and `/barry/dataset/dataset_correlation_function.py`. These are currently coded up to only read in the LRG samples above, but again can be extended easily enough for other tracers.

The way you get the data is by instantiating the class, with an appropriate set of arguments. Some of these arguments determine how the data will be treated. The others tell the code which of the files above to read in. The full set of (accessible) arguments for the power spectrum is:

* `min_k`: The minimum k-value to consider. Default is 0.02
* `max_k`: The maximum k-value to consider. Default is 0.30
* `recon`: Whether to use pre- or post-reconstruction data. Choices are `None`, "iso", "ani" or "sym", but only `None` is currently implemented. Default is `None`
* `reduce_cov_factor`: Factor to divide the covariance matrix by. Default is 1.0
* `realisation`: The realisation to load. Choices are "data", `None` (mock mean), or an integer (mock realisation). Default is `None`.
* `fit_poles`: A list of multipoles to consider in the fitting. Default is `(0,)`.
* `mocktype`: Which KP4 mocks to load. Choices are "abacus_cubicbox", "abacus_cutsky", "ezmock_cubicbox", "ezmock_cutsky". Default is "abacus_cubicbox". **All files use the EZmock covariance matrix, but are otherwise separated to give access to the individual realisations for each mock type.**.
* `num_mocks`: The number of mocks used to estimate the covariance matrix, for HARTLAP or SELLENTIN corrections. Default is 1000. **This is correct for all mocks, even abacus, as we use the EZmock covariance matrices**.
* `fake_diag`: If True, use only the diagonals of the covariance matrix. Default is False.
* `tracer`: Which tracer to load. Only "lrg" is currently implemented, which is the default.
* `redshift_bin`: Which redshift bin to load. Choices change based on `tracer` and `mocktype`. Default is 1.
    * `tracer=lrg`.
        * 1 for `mocktype="abacus_cubicbox"` or `mocktype="ezmock_cubicbox"`. 
        * 1, 2 or 3 for `mocktype="abacus_cutsky"` or `mocktype="ezmock_cutsky"`.
        
The arguments are the same for the correlation function, except for `min_dist` and `max_dist` instead of `min_k` and `max_k`.
        
The only real relevant functions for the class are:
* `get_data()`: Return a list of dictionaries for the data
* `set_realisation()`: Useful for changing the realisation you are using without re-initialising the class.

Let's try these out and plot some data

In [ ]:
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4

# Lets read in a single instance of the data and have a look
dataset = PowerSpectrum_DESI_KP4(mocktype="abacus_cutsky")
data = dataset.get_data()[0]
print(data)

# Lets read in a single instance of the data and have a look
dataset = CorrelationFunction_DESI_KP4(mocktype="ezmock_cubicbox")
data = dataset.get_data()[0]
print(data)

This prints out a lot of information about the dataset! Most of it is just useful for the fitting (i.e., window functions, inverse covariance matrices, masking the appropriate fitting ranges), but we might want to access some of it directly. For instance, we can plot the data using the "ks", "cov" and "pk0", "pk2", "pk4" entries. For correlation function we can use "s", "cov", "xi0", "xi2" and "xi4". The "num_mocks" entry and "cosmology" (the fiducial cosmology) might be useful too.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_pk(dataset):
    
    color = ["r", "b", "g"]
    label = [r"$P_{0}(k)$", r"$P_{2}(k)$", r"$P_{4}(k)$"]
    
    # Plot some pk data
    for m, pk in enumerate(["pk0", "pk2", "pk4"]):
        dataset.set_realisation(None) # Sets the data to the mock mean.
        data = dataset.get_data()[0]
        ks, err = data["ks"], np.sqrt(np.diag(data["cov"]))
        yerr = ks * err[2 * m * len(ks) : (2 * m + 1) * len(ks)]
        plt.errorbar(
            ks,
            ks * data[pk][0],
            yerr=yerr,
            marker="o",
            ls="None",
            c=color[m],
            label=label[m],
        )
        for i in range(len(dataset.mock_data)):
            dataset.set_realisation(i)  # Changes the realisation to one of the mocks, and plots that.
            data = dataset.get_data()[0]
            plt.errorbar(ks, ks * data[pk][0], marker="None", ls="-", c="k", alpha=1.0 / len(dataset.mock_data)**(3.0/4.0))
    plt.xlabel(r"$k$")
    plt.ylabel(r"$k\,P(k)$")
    plt.title(dataset.name)
    plt.legend(loc='upper right')
    plt.show()
    
def plot_xi(dataset):
    
    color = ["r", "b", "g"]
    label = [r"$\xi_{0}(k)$", r"$\xi_{2}(k)$", r"$\xi_{4}(k)$"]
    
    # Plot sxome xi data
    for m, xi in enumerate(["xi0", "xi2", "xi4"]):
        dataset.set_realisation(None) # Sets the data to the mock mean.
        data = dataset.get_data()[0]
        ss, err = data["dist"], np.sqrt(np.diag(data["cov"]))
        yerr = ss ** 2 * err[m * len(ss) : (m + 1) * len(ss)]
        plt.errorbar(
            ss,
            ss ** 2 * data[xi],
            yerr=yerr,
            marker="o",
            ls="None",
            c=color[m],
            label=label[m],
        )
        for i in range(len(dataset.mock_data)):
            dataset.set_realisation(i)  # Changes the realisation to one of the mocks, and plots that.
            data = dataset.get_data()[0]
            plt.errorbar(ss, ss ** 2 * data[xi], marker="None", ls="-", c='k', alpha=1.0 / len(dataset.mock_data)**(3.0/4.0))
    plt.xlabel(r"$s$")
    plt.ylabel(r"$s^{2}\,\xi(s)$")
    plt.title(dataset.name)
    plt.legend(loc='upper right')
    plt.show()
    
dataset = PowerSpectrum_DESI_KP4()
plot_pk(dataset)

dataset = CorrelationFunction_DESI_KP4()
plot_xi(dataset)

In [ ]:
# Let's try a different k-range, mock type and redshift bin
dataset = PowerSpectrum_DESI_KP4(mocktype="abacus_cutsky", min_k=0.0, max_k=0.25, redshift_bin=1)
plot_pk(dataset)

# Let's try a different s-range, mock type and redshift bin
dataset = CorrelationFunction_DESI_KP4(mocktype="abacus_cutsky", min_dist=50.0, max_dist=170.0, redshift_bin=3)
plot_xi(dataset)

In [ ]:
# Let's try all the possible tracers, mocktypes and redshift bins
tracers = ["lrg", "lrg", "lrg", "lrg"]
nzbins = [1, 3, 1, 3]
mocktypes = ["abacus_cubicbox", "abacus_cutsky", "ezmock_cubicbox", "ezmock_cutsky"]
for j, (tracer, redshift_bins, mocktype) in enumerate(zip(tracers, nzbins, mocktypes)):
    for z in range(redshift_bins):
        plot_pk(PowerSpectrum_DESI_KP4(
            recon=None,
            fit_poles=[0, 2, 4],
            min_k=0.02,
            max_k=0.30,
            mocktype=mocktype,
            tracer=tracer,
            redshift_bin=z + 1,
            realisation=None,
        ))
        plot_xi(CorrelationFunction_DESI_KP4(
            recon=None,
            fit_poles=[0, 2, 4],
            min_dist=30.0,
            max_dist=200.0,
            mocktype=mocktype,
            tracer=tracer,
            redshift_bin=z + 1,
            realisation=None,
        ))

# Models
Okay. Now we have a handle on the datasets, and how to load them in, let's see about running a fit. The first thing we need to do is choose a model. Barry currently has the following options:
* `bao_power_Beutler2017`: Model as in Beutler et. al., 2017 with free damping parameters
* `bao_power_Chen2019`: Model as in Chen et. al., 2019 with Zeldovich approximation based BAO damping
* `bao_power_Ding2018`: Model as in Ding et. al., 2018 with EFT-based model
* `bao_power_Seo2016`: Model as in Seo et. al., 2016 with LPT inspired BAO damping.
* `bao_correlation_Beutler2017`: Correlation function transformation applied to Beutler et. al., 2017 power spectrum with free damping parameters
* `bao_correlation_Ding2018`: Correlation function transformation applied to Ding et. al., 2018 power spectrum with EFT-based model
* `bao_correlation_Seo2016`: Correlation function transformation applied to Seo et. al., 2016 power spectrum with LPT inspired BAO damping.

Each of these also has different free parameters that can be allowed to vary or be fixed. Power spectra support 0, 3 or 5 polynomial terms added to each multipole, correlation function is currently just 3. 

The models work as classes, and have the following (relevant) keyword arguments:
* `fix_params`: A list of parameters to fix in the model.
* `recon`: A reconstruction convention, same options as for data. Easiest to set as recon=dataset.recon.
* `smooth`: Whether to return a model with (`False`) or without (`True`) BAO wiggles. Default is `True`.
* `correction:` A correction to use for the likelihood to account for the errors in the covariance matrix. Corrections are also classes, but typically we'll want Correction.NONE, Correction.HARTLAP or Correction.SELLENTIN.
* `isotropic`: Whether to fit for only alpha, or for alpha and epsilon.
* `poly_poles`: Which multipoles to add polynomial terms to.
* `marg`: Whether to speed things up by analytically marginalising over linear order broadband terms. Options are `None`, "full" (proper Bayesian marginalisation) or "partial" (find the best-fit polynomial terms at each point in the MCMC chain).
* `dilate_smooth`: For the Beutler 2017 power spectrum model only, whether to dilate only the wiggles of the model (`False`), or the whole model (`True`), by the BAO dilation parameters. Default is True.
* `n_poly`: Only for power spectrum. The number of polynomial terms per multipole. Default is 5.

So lets have a look at creating and plotting some models! Useful functions are:
* `get_defaults()`: Return a set of default parameter values.
* `set_default(param, value)`: Update the default value for a parameter.
* `get_param_dict(params)`: Set the values of the parameters to the values in params and return a dictionary.
* `set_cosmology(cosmo)`: Set the template cosmology given a dictionary of cosmology parameters.
* `compute_power_spectrum(k, param_dict)`: Compute the model for a dictionary of parameter values.
* `set_data(dataset)`: Set the model up to return a prediction for some data. Set's up the template cosmology to the fiducial cosmology of the data, organises the k-binning and window function convolution for you.

In [ ]:
from barry.models.bao_power_Beutler2017 import PowerBeutler2017

# Instantiate a model.
model_iso = PowerBeutler2017(isotropic=True)
model_ani = PowerBeutler2017(isotropic=False)

# Get the default values of the model and also as a dictionary. You should see that the
# anisotropic model has a lot more polynomial terms and also separate BAO damping perpendicular and parallel
# to the line of sight. This is because it fits the multipoles, rather than just the monopole.
p_dict_iso = model_iso.get_param_dict(model_iso.get_defaults())
p_dict_ani = model_ani.get_param_dict(model_ani.get_defaults())
print(p_dict_iso, p_dict_ani)

# Let's see some other combos. Include polynomials for hexadecapole:
model = PowerBeutler2017(isotropic=False, poly_poles=(0,2,4)) 
# Fix the BAO damping parameters and beta=f/b to their default values:
model = PowerBeutler2017(isotropic=False, poly_poles=(0,2,4), fix_params=("sigma_nl_perp", "sigma_nl_par", "beta"))

# Update the default values of the sigma_nl's. The default is 4 and 8 for perp and par respectively.
# You can see these get updated to 5 and 11.
print(model.get_param_dict(model.get_defaults()))
model.set_default("sigma_nl_perp", 5.0)
model.set_default("sigma_nl_par", 11.0)
print(model.get_param_dict(model.get_defaults()))

To return the actual model prediction, we need to first tell the model what the template cosmology is, then compute it. We'll start with unmarginalised models, as these return the actual model, rather than components. For fitting purposes it's better to use `marg="full"`, but it's less interpretable what the model is doing

In [ ]:
from barry.models.bao_power_Ding2018 import PowerDing2018

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
        }
model = PowerDing2018(isotropic=False)
model.set_cosmology(cosmology)
ks = model.camb.ks

# This function returns the values of k'(k,mu), pk (technically only the unmarginalised terms)
# and model components for analytically marginalised parameters
p = model.get_param_dict(model.get_defaults())
kprime, pk, marged = model.compute_power_spectrum(ks, p)

# Plot the model multipoles
plt.errorbar(ks, ks * pk[0], marker="None", ls="-", c="r", label=r"$P_{0}(k)$")
plt.errorbar(ks, ks * pk[2], marker="None", ls="-", c="b", label=r"$P_{2}(k)$")
plt.errorbar(ks, ks * pk[4], marker="None", ls="-", c="g", label=r"$P_{4}(k)$")

# Update alpha and epsilon, and show a comparison
p["alpha"] = 1.05
p["epsilon"] = -0.04
kprime, pk_new, marged = model.compute_power_spectrum(ks, p)

# Plot the model multipoles
plt.errorbar(ks, ks * pk_new[0], marker="None", ls="--", c="r", label=r"$P_{0}(k)$")
plt.errorbar(ks, ks * pk_new[2], marker="None", ls="--", c="b", label=r"$P_{2}(k)$")
plt.errorbar(ks, ks * pk_new[4], marker="None", ls="--", c="g", label=r"$P_{4}(k)$")
plt.xlim(0.0, 0.3)
plt.show()

## Including the window function and data binning
To include the window function/wide angle matrix (for power spectra) and data binning we need to tell the model the properties of the data. We do this by loading a dataset, running `get_data()` on this, and then `set_data()` for the model. Setting the data for the model also sets the template cosmology of the model to the fiducial cosmology of the data.

In [ ]:
# Load in a dataset
dataset = PowerSpectrum_DESI_KP4(mocktype="abacus_cutsky", fit_poles=(0,2,4), min_k=0.02, max_k=0.25, redshift_bin=1)
data = dataset.get_data()

# Set up a model and tell it about the data
model = PowerBeutler2017(isotropic=False, poly_poles=(0,2), marg=None)
model.set_data(data)

# Get the model without window function convolution
p = model.get_param_dict(model.get_defaults())
kprime, pk, marged = model.compute_power_spectrum(ks, p)

# And now with window function convolution
kout = data[0]["ks_output"]
pk_win, pk_win_odd, marged_win, marged_win_odd, mask = model.get_model(p, data[0])

# Plot the convolved model multipoles
plt.errorbar(ks, ks * pk[0], marker="None", ls="-", c="r", label=r"$P_{0}(k)$")
plt.errorbar(ks, ks * pk[2], marker="None", ls="-", c="b", label=r"$P_{2}(k)$")
plt.errorbar(ks, ks * pk[4], marker="None", ls="-", c="g", label=r"$P_{4}(k)$")
plt.errorbar(kout, kout * pk_win[:len(kout)], marker="None", ls="--", c="r")
plt.errorbar(kout, kout * pk_win[2*len(kout):3*len(kout)], marker="None", ls="--", c="b")
plt.errorbar(kout, kout * pk_win[4*len(kout):5*len(kout)], marker="None", ls="--", c="g")
plt.xlim(0.0, 0.3)
plt.show()

# Odd multipoles
plt.errorbar(kout, kout * pk_win[1*len(kout):2*len(kout)], marker="None", ls="--", c="r")
plt.errorbar(kout, kout * pk_win[3*len(kout):4*len(kout)], marker="None", ls="--", c="b")
plt.xlim(0.0, 0.3)
plt.show()

# And now masked to the data fitting ranges
kmasked = data[0]["ks"]
pk_masked, pk_masked_odd = pk_win[mask], pk_win_odd[mask]

# Model multipoles at data bins we are including in the fit
plt.errorbar(ks, ks * pk[0], marker="None", ls="-", c="r", label=r"$P_{0}(k)$")
plt.errorbar(ks, ks * pk[2], marker="None", ls="-", c="b", label=r"$P_{2}(k)$")
plt.errorbar(ks, ks * pk[4], marker="None", ls="-", c="g", label=r"$P_{4}(k)$")
plt.errorbar(kmasked, kmasked * pk_masked[:len(kmasked)], marker="None", ls="--", c="r")
plt.errorbar(kmasked, kmasked * pk_masked[len(kmasked):2*len(kmasked)], marker="None", ls="--", c="b")
plt.errorbar(kmasked, kmasked * pk_masked[2*len(kmasked):3*len(kmasked)], marker="None", ls="--", c="g")
plt.xlim(0.0, 0.3)
plt.show()

# Fitting the data
Okay, now let's do some proper fits. We first set the data, then we can run any fits we want

In [ ]:
# We can import in this way too as everything is indexed in the relevant __init__.py files
from barry.datasets import PowerSpectrum_DESI_KP4, CorrelationFunction_DESI_KP4
from barry.models import PowerBeutler2017, PowerDing2018, CorrBeutler2017, CorrSeo2016

# Load in the z=0.4-0.6 ezmock_cutsky dataset with and without the hexadecapole included in the fit
pk_dataset = PowerSpectrum_DESI_KP4(mocktype="abacus_cubicbox", fit_poles=(0,2), min_k=0.02, max_k=0.30, redshift_bin=1)
pk_dataset_hexa = PowerSpectrum_DESI_KP4(mocktype="abacus_cubicbox", fit_poles=(0,2,4), min_k=0.02, max_k=0.20, redshift_bin=1)
xi_dataset = CorrelationFunction_DESI_KP4(mocktype="abacus_cubicbox", fit_poles=(0,2), min_dist=35.0, max_dist=170.0, redshift_bin=1)
xi_dataset_hexa = CorrelationFunction_DESI_KP4(mocktype="abacus_cubicbox", fit_poles=(0,2,4), min_dist=50.0, max_dist=160.0, redshift_bin=1)

# A random selection of model choices
pk_model_ding = PowerDing2018(isotropic=False, poly_poles=(0,2), marg="full")
pk_model_beutler = PowerBeutler2017(isotropic=False, poly_poles=(0,2,4), marg="full", n_poly=3, dilate_smooth=False)
xi_model_beutler = CorrSeo2016(isotropic=False, poly_poles=(0,2), marg="full")
xi_model_seo = CorrBeutler2017(isotropic=False, poly_poles=(0,2,4), marg="full")

# Loop over all the model and appropriate dataset and run an opimisation on each one.
names = ["pk_model_ding", "pk_model_beutler", "xi_model_beutler", "xi_model_seo"]
datasets = [pk_dataset, pk_dataset_hexa, xi_dataset, xi_dataset_hexa]
models = [pk_model_ding, pk_model_beutler, xi_model_beutler, xi_model_seo]
for i, (name, dataset, model) in enumerate(zip(names, datasets, models)):
    data = dataset.get_data()[0]
    model.set_data(dataset.get_data())
    print("Starting model optimisation. This may take some time.")
    p, maxlike = model.optimize()
    print(name)
    print("alpha, epsilon:", p["alpha"], p["epsilon"])
    print("alpha_par, alpha_perp:", model.get_alphas(p["alpha"], p["epsilon"]))
    print("-------------")
    # A convenient plotting function for the best-fit, very useful as pk/xi from compute_power_spectrum or 
    # compute_correlation_function is tough to interpret when using analytic marginalisation
    model.plot(p)

# the model.plot function can also be used to return useful things without the plot -- 
# chi2, degrees-of-freedom, best-fit values of any analytically marginalised parameters, 
# the best-fit model multipoles, and the best-fit smooth model multipoles
new_chi_squared, dof, bband, mods, smooths = model.plot(p, display=False)

# The previous set of functions to fit a model to data are also wrapped up in a convenient `sanity_check()` function.
model.sanity_check(dataset)

And there we go! To run proper MCMC fits to the data, we also need to set up a sampler. There are currently the following scripts in the repo to fit all the KP4 data
* /config/desi_kp4/desi_kp4_abacus_cubic_pk.py
* /config/desi_kp4/desi_kp4_abacus_cubic_xi.py
* /config/desi_kp4/desi_kp4_abacus_pk.py
* /config/desi_kp4/desi_kp4_abacus_xi.py

Each one can be run as a python code from the login node on NERSC (assuming you have set your environment as in the NERSC.md), i.e., `python desi_kp4_abacus_cubic_pk.py`. This will prepare and submit all the jobs for you, fitting the mock mean and 25 realisations for each redshift.

Once the chains have finished running, if you the code again using i.e., `python desi_kp4_abacus_cubic_pk.py plot` it will read in the existing chains and make summary plots and files.